In [0]:
from pyspark.sql.functions import regexp_extract, col

commentary_df = spark.read.format("csv") \
    .option("header", "true") \
    .load("dbfs:/FileStore/commentary_csv/part-00000-tid-*.csv")


commentary_df = commentary_df.withColumn("bowler", regexp_extract(col("commentary"), r"^(\w+)", 1)) \
                             .withColumn("batsman", regexp_extract(col("commentary"), r"to (\w+)", 1))


commentary_df = commentary_df.withColumn("runs", 
    regexp_extract(col("commentary"), r"(FOUR|four|SIX|six|no run|1 run|2 runs|3 runs|OUT)", 1))

from pyspark.sql.functions import when

commentary_df = commentary_df.withColumn("runs",
    when(col("runs").rlike("(?i)four"), 4)
    .when(col("runs").rlike("(?i)six"), 6)
    .when(col("runs").rlike("(?i)1 run"), 1)
    .when(col("runs").rlike("(?i)2 runs"), 2)
    .when(col("runs").rlike("(?i)3 runs"), 3)
    .when(col("runs").rlike("(?i)no run|OUT"), 0)
    .otherwise(None)
)

commentary_df.select("commentary", "bowler", "batsman", "runs").show(truncate=False)


+-------------------+------+-------+----+
|commentary         |bowler|batsman|runs|
+-------------------+------+-------+----+
|Dhoni hits a six   |Dhoni |       |6   |
|Rohit scores a four|Rohit |       |4   |
|Kohli gets bowled  |Kohli |       |null|
+-------------------+------+-------+----+



In [0]:
from pyspark.sql.functions import col, regexp_extract

df = spark.read.csv("dbfs:/FileStore/commentary_csv/part-00000*.csv", header=True)

df = df.withColumn("batsman", regexp_extract("commentary", r"^(\w+)", 1))

df = df.withColumn("runs",
    regexp_extract("commentary", r".*?(\bsix\b|\bfour\b|\b1 run\b|\b2 runs\b|\b3 runs\b|\bno run\b)", 1)
)

from pyspark.sql.functions import when

df = df.withColumn("runs",
    when(col("runs") == "six", 6)
    .when(col("runs") == "four", 4)
    .when(col("runs") == "1 run", 1)
    .when(col("runs") == "2 runs", 2)
    .when(col("runs") == "3 runs", 3)
    .when(col("runs") == "no run", 0)
    .otherwise(None)
)

df.select("commentary", "batsman", "runs").show(truncate=False)


+-------------------+-------+----+
|commentary         |batsman|runs|
+-------------------+-------+----+
|Dhoni hits a six   |Dhoni  |6   |
|Rohit scores a four|Rohit  |4   |
|Kohli gets bowled  |Kohli  |null|
+-------------------+-------+----+

